<a href="https://colab.research.google.com/github/Nanxi-lu/BSA_Big_data/blob/main/%E2%80%9CW05_quiz_2026_ipynb%E2%80%9D%E7%9A%84%E5%89%AF%E6%9C%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Week 5 Quiz

This notebook contains the SQL Quiz for Week 5. Section 1 uses the New York City data we know and love. Section 2 uses building footprint data from the Google Open Buildings dataset.

INSTRUCTIONS:

Run this notebook in Google Colab. The answer to each quesiton will be a number or a string. Input these into the corresponding question on Moodle. You have 90 minutes to attempt the quiz, so if you get stuck on a question, move on.

Make sure you run all of the cells of code in order, especially the ones that already have code in them! If you run into serious problems, try clicking on the "runtime" tab above and selecting "restart session and run all".
Use the cells below for any necessary setup.

In [ ]:
import os
from pathlib import Path

In [ ]:
%pip install duckdb leafmap

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 667.8/667.8 kB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.4/20.4 MB 75.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 99.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 63.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/41.8 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.6/108.6 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 89.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.5/208.5 kB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.0/74.0 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 66.6 MB/s eta 0:00:00
  Attempting uninstall: duckdb
    Found existing installation: duckdb 1.3.2
    Uninstalling duckdb-1.3.2:
      Successfully uninstalled duckdb-1.3.2


In [ ]:
import duckdb
import leafmap

In [ ]:
data_dir = Path("nyc_data")
zip_path = Path("postgis-workshop-2020.zip")
data_dir.mkdir(exist_ok=True)

# Section 1



The URL below points to a zip file containing the data on New York City that we've been working with so far.

https://s3.amazonaws.com/s3.cleverelephant.ca/postgis-workshop-2020.zip

In [ ]:
# zip
url = "https://s3.amazonaws.com/s3.cleverelephant.ca/postgis-workshop-2020.zip"
leafmap.download_file(url, str(zip_path))

Downloading...
From: https://s3.amazonaws.com/s3.cleverelephant.ca/postgis-workshop-2020.zip
To: /content/postgis-workshop-2020.zip
100%|██████████| 22.5M/22.5M [00:00<00:00, 29.0MB/s]


Extracting files...


'/content/postgis-workshop-2020.zip'

In [ ]:
import zipfile

In [ ]:
# Unzip using standard Python
with zipfile.ZipFile(zip_path, "r") as z:
    z.extractall(data_dir)

In [ ]:
# Connect to DuckDB and enable spatial
db_path = "nyc.duckdb"
con = duckdb.connect(db_path)

con.install_extension("spatial")
con.load_extension("spatial")

con.sql("SELECT 1;").fetchall()


[(1,)]

In [ ]:
# shapefile
def find_shp(keyword: str) -> str:
    matches = [p for p in data_dir.rglob("*.shp") if keyword.lower() in p.name.lower()]
    if not matches:
        raise FileNotFoundError(f"No .shp found for keyword: {keyword}")
    # If there are multiple matches, pick the shortest path (usually the main file)
    matches = sorted(matches, key=lambda p: (len(str(p)), str(p)))
    return str(matches[0])

shp_neighborhoods = find_shp("neighborhood")
shp_census_blocks = find_shp("census")     # sometimes "census_blocks" or similar
shp_homicides = find_shp("homicide")
shp_streets = find_shp("street")
shp_subway = find_shp("subway")

## Question 1

Create the following tables using the corresponding shapefiles.
- nyc_neighborhoods
- nyc_census_blocks
- nyc_homicides
- nyc_streets
- nyc_subway_stations

In [ ]:
# Create tables
con.sql(f"""
CREATE OR REPLACE TABLE nyc_neighborhoods AS
SELECT * FROM ST_Read('{shp_neighborhoods}');
""")

con.sql(f"""
CREATE OR REPLACE TABLE nyc_census_blocks AS
SELECT * FROM ST_Read('{shp_census_blocks}');
""")

con.sql(f"""
CREATE OR REPLACE TABLE nyc_homicides AS
SELECT * FROM ST_Read('{shp_homicides}');
""")

con.sql(f"""
CREATE OR REPLACE TABLE nyc_streets AS
SELECT * FROM ST_Read('{shp_streets}');
""")

con.sql(f"""
CREATE OR REPLACE TABLE nyc_subway_stations AS
SELECT * FROM ST_Read('{shp_subway}');
""")


In [ ]:
con.sql("""
SHOW TABLES;
""").fetchdf()

,name
0,nyc_census_blocks
1,nyc_homicides
2,nyc_neighborhoods
3,nyc_streets
4,nyc_subway_stations


In [ ]:
con.sql("""
SELECT table_name, column_name, data_type
FROM information_schema.columns
WHERE table_name IN (
  'nyc_neighborhoods',
  'nyc_census_blocks',
  'nyc_homicides',
  'nyc_streets',
  'nyc_subway_stations'
)
ORDER BY table_name, ordinal_position;
""").fetchdf()


,table_name,column_name,data_type
0,nyc_census_blocks,BLKID,VARCHAR
1,nyc_census_blocks,POPN_TOTAL,BIGINT
2,nyc_census_blocks,POPN_WHITE,BIGINT
3,nyc_census_blocks,POPN_BLACK,BIGINT
4,nyc_census_blocks,POPN_NATIV,BIGINT
5,nyc_census_blocks,POPN_ASIAN,BIGINT
6,nyc_census_blocks,POPN_OTHER,BIGINT
7,nyc_census_blocks,BORONAME,VARCHAR
8,nyc_census_blocks,geom,GEOMETRY
9,nyc_homicides,INCIDENT_D,DATE


## Question 2:
How many subway stations are there in NYC?



In [ ]:
con.sql("""
SELECT
  (SELECT COUNT(*) FROM nyc_neighborhoods) AS neighborhoods_n,
  (SELECT COUNT(*) FROM nyc_census_blocks) AS census_blocks_n,
  (SELECT COUNT(*) FROM nyc_homicides) AS homicides_n,
  (SELECT COUNT(*) FROM nyc_streets) AS streets_n,
  (SELECT COUNT(*) FROM nyc_subway_stations) AS subway_stations_n;
""").fetchdf()

,neighborhoods_n,census_blocks_n,homicides_n,streets_n,subway_stations_n
0,129,38794,3984,19091,491


## Question 3:

What combination of borough and year hard the largest number of homicide victims?

In [ ]:
con.sql("""
SELECT
  boroname,
  year,
  COUNT(*) AS homicide_count
FROM nyc_homicides
GROUP BY boroname, year
ORDER BY homicide_count DESC
LIMIT 1;
""").fetchdf()


,BORONAME,YEAR,homicide_count
0,Brooklyn,2003,218


## Question 4:

Which subway station has the largest number of shooting victims within a 300 meter radius?


In [ ]:
con.sql("""
SELECT
  s.name AS station_name,
  SUM(CAST(h.num_victim AS INTEGER)) AS victim_count
FROM nyc_subway_stations AS s
JOIN nyc_homicides AS h
ON ST_DWithin(
  s.geom,
  h.geom,
  300
)
GROUP BY s.name
ORDER BY victim_count DESC
LIMIT 1;
""").fetchdf()


,station_name,victim_count
0,Church Ave,33.0


## Question 5

what is the most densely populated residential street in new york? (total population of intersecting census blocks divided by the total length of the street)

In [ ]:
con.sql("""
SELECT
  type,
  COUNT(*) AS n
FROM nyc_streets
GROUP BY type
ORDER BY n DESC;
""").fetchdf()


,TYPE,n
0,residential,16560
1,motorway_link,953
2,footway,345
3,unclassified,337
4,tertiary,257
5,motorway,227
6,secondary,137
7,primary,98
8,service,90
9,pedestrian,20


In [ ]:
con.sql("""
WITH street_pop AS (
  SELECT
    s.id,
    s.name AS street_name,
    SUM(cb.popn_total) AS total_population,
    ST_Length(s.geom) AS street_length_m
  FROM nyc_streets AS s
  JOIN nyc_census_blocks AS cb
    ON ST_Intersects(s.geom, cb.geom)
  WHERE s.type = 'residential'
  GROUP BY s.id, s.name, s.geom
),
street_density AS (
  SELECT
    id,
    street_name,
    total_population / street_length_m AS pop_per_meter
  FROM street_pop
  WHERE street_length_m > 0
)
SELECT
  street_name,
  pop_per_meter
FROM street_density
ORDER BY pop_per_meter DESC
LIMIT 3;
""").fetchdf()

,street_name,pop_per_meter
0,W 66 St,3223.787480
1,Rikers Is Br,422.560822
2,Eagle N Ln,356.255957


# Section 2

This section uses internet speed data from [Ookla](https://github.com/teamookla/ookla-open-data); take a minute to read the documentation of the dataset.

Create a table called ookla_nyc which reads the data from this url:

https://storage.googleapis.com/qm2/CASA0025/ookla_nyc.parquet




In [ ]:
con.sql("""
CREATE OR REPLACE TABLE ookla_nyc AS
SELECT *
FROM read_parquet(
  'https://storage.googleapis.com/qm2/CASA0025/ookla_nyc.parquet'
);
""")

In [ ]:
con.sql("""
SHOW TABLES;
""").fetchdf()

,name
0,nyc_census_blocks
1,nyc_homicides
2,nyc_neighborhoods
3,nyc_streets
4,nyc_subway_stations
5,ookla_nyc


## Question 6
How many tiles are contained within new york city neighbourhoods?

In [ ]:
con.sql("""
DESCRIBE ookla_nyc;
""").fetchdf()

,column_name,column_type,null,key,default,extra
0,quadkey,VARCHAR,YES,None,None,None
1,tile_x,DOUBLE,YES,None,None,None
2,tile_y,DOUBLE,YES,None,None,None
3,avg_d_kbps,BIGINT,YES,None,None,None
4,avg_u_kbps,BIGINT,YES,None,None,None
5,avg_lat_ms,BIGINT,YES,None,None,None
6,avg_lat_down_ms,INTEGER,YES,None,None,None
7,avg_lat_up_ms,INTEGER,YES,None,None,None
8,tests,BIGINT,YES,None,None,None
9,devices,BIGINT,YES,None,None,None


In [ ]:
con.sql("""
CREATE OR REPLACE TABLE ookla_nyc_geom AS
SELECT
  *,
  ST_GeomFromText(geometry) AS geom
FROM ookla_nyc;
""")

In [ ]:
con.sql("""
SELECT
  COUNT(*) AS tile_count
FROM ookla_nyc_geom AS o
JOIN nyc_neighborhoods AS n
ON ST_Within(o.geom, n.geom);
""").fetchdf()

,tile_count
0,1403


## Question 7

Which neighbourhood has the highest average download speed? use 'within' rather than intersects, and ignore tiles with fewer than 3 tests.

In [ ]:
con.sql("""
SELECT
  n.name AS neighborhood,
  AVG(o.avg_d_kbps) AS avg_download_kbps
FROM ookla_nyc_geom AS o
JOIN nyc_neighborhoods AS n
ON ST_Within(o.geom, n.geom)
WHERE o.tests >= 3
GROUP BY n.name
ORDER BY avg_download_kbps DESC
LIMIT 2;
""").fetchdf()

,neighborhood,avg_download_kbps
0,Woodside,455960.333333
1,Glendale,430165.777778


## Question 8



What is the name of the subway station closest to the tile with the highest download speed?

In [ ]:
con.sql("""
WITH fastest_tile AS (
  SELECT
    geom
  FROM ookla_nyc_geom
  WHERE tests >= 3
  ORDER BY avg_d_kbps DESC
  LIMIT 1
)
SELECT
  s.name AS station_name,
  ST_Distance(s.geom, f.geom) AS distance_m
FROM nyc_subway_stations AS s
CROSS JOIN fastest_tile AS f
ORDER BY distance_m ASC;
""").fetchdf()

,station_name,distance_m
0,Nassau,9129.213699
1,Richmond Valley,9200.404652
2,Atlantic,9264.386813
3,Pleasant Plains,9299.457346
4,Tottenville,9500.923332
...,...,...
486,233rd St,47434.311672
487,Baychester Ave,47530.984862
488,Nereid Ave,48005.001922
489,241st St,48625.251416


## Question 9

Which neighbourhood has the highest number of devices per capita? Use only spatial intersections.

In [ ]:
con.sql("""
WITH devices_by_neighborhood AS (
  SELECT
    n.name AS neighborhood,
    SUM(o.devices) AS devices_total
  FROM nyc_neighborhoods AS n
  JOIN ookla_nyc_geom AS o
    ON ST_Intersects(o.geom, n.geom)
  GROUP BY n.name
),
pop_by_neighborhood AS (
  SELECT
    n.name AS neighborhood,
    SUM(cb.popn_total) AS pop_total
  FROM nyc_neighborhoods AS n
  JOIN nyc_census_blocks AS cb
    ON ST_Intersects(cb.geom, n.geom)
  GROUP BY n.name
)
SELECT
  d.neighborhood,
  d.devices_total,
  p.pop_total,
  d.devices_total * 1.0 / p.pop_total AS devices_per_capita
FROM devices_by_neighborhood AS d
JOIN pop_by_neighborhood AS p
  ON d.neighborhood = p.neighborhood
WHERE p.pop_total > 0
ORDER BY devices_per_capita DESC;
""").fetchdf()

,neighborhood,devices_total,pop_total,devices_per_capita
0,Corona,162.0,333.0,0.486486
1,Garment District,2253.0,10539.0,0.213777
2,Fresh Kills,21.0,100.0,0.210000
3,Rosedale,10.0,111.0,0.090090
4,Downtown,1047.0,11643.0,0.089925
...,...,...,...,...
118,Tremont,916.0,122743.0,0.007463
119,Morris Heights,547.0,73413.0,0.007451
120,The Rockaways,770.0,112535.0,0.006842
121,Springfield Gardens,272.0,41157.0,0.006609


## Question 10


What is the average upload speed for tiles that are within 200 meters of L train stops and intersect with census blocks where the black population is above 50%?

In [ ]:
con.sql("""
WITH l_train_stations AS (
  SELECT
    geom
  FROM nyc_subway_stations
  WHERE routes LIKE '%L%'
),
high_black_blocks AS (
  SELECT
    geom
  FROM nyc_census_blocks
  WHERE popn_black * 1.0 / popn_total > 0.5
)
SELECT
  AVG(o.avg_u_kbps) AS avg_upload_kbps
FROM ookla_nyc_geom AS o
JOIN l_train_stations AS l
  ON ST_DWithin(o.geom, l.geom, 200)
JOIN high_black_blocks AS b
  ON ST_Intersects(o.geom, b.geom);
""").fetchdf()

,avg_upload_kbps
0,124413.306954
